# Boilerplate

In [ ]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [ ]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [ ]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [ ]:
client_tiface = await new_transport_interface("0.0.0.0", 0, TransportModeW("client"))
client_tiface.run()

server_tiface = await new_transport_interface("0.0.0.0", SERVER_PORT, TransportModeW("server"))
server_tiface.run()

# Filter layer setup

In [ ]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

Find methods to run, for informational purposes.

In [ ]:
[m for m in dir(client_fiface) if not m.startswith('__')]

### Run Client and Server Filter!

In [ ]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above futures should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

## Filter layer operation for Pings

### Change the server status to something meaningful

In [ ]:
server_name = "Local Conwayste Server (notebook)"
server_version = "1.2.34.56"
player_count = 8675309
room_count = 9001
await server_fiface.command_response(FilterCmdW("changeserverstatus",
                                                server_name=server_name,
                                                server_version=server_version,
                                                player_count=player_count,
                                                room_count=room_count))

### As a client, add server (both our local and Chococat) as ping endpoints

#### First, resolve DNS for Chococat.

In [ ]:
import socket
choco_ip = socket.gethostbyname_ex('chococat.conwayste.rs')[-1][0]
choco_ip

In [ ]:
server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")
choco_ept  = EndpointW(f"{choco_ip}:2016")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[server_ept, choco_ept])
filter_cmd

In [ ]:
await client_fiface.command_response(filter_cmd)

The client will repeatedly send pings. Need `LATENCY_FILTER_DEPTH` pings (currently 4) to be received back from server before we know the latency.

### Sleep for a bit

We need time to get some pings.

In [ ]:
time.sleep(3.0)

### Clear ping endpoints

In [ ]:
await client_fiface.command_response(FilterCmdW("clearpingendpoints"))

### Get notifications from client and server

Normally the App layer would be getting these continuously.

In [ ]:
server_notifications = server_fiface.get_notifications()
server_notifications

In [ ]:
client_notifications = client_fiface.get_notifications()
len(client_notifications)

In [ ]:
list(map(lambda n: n.variant, client_notifications))

In [ ]:
def server_info(notif):
    assert notif.variant.lower() == 'pingresult'
    return {'latency_ms':notif.latency, 'server_name': notif.server_name, 'room_count': notif.room_count, 'REPR': repr(notif)}

In [ ]:
server_info(client_notifications[-1])

In [ ]:
server_info(client_notifications[-2])

Sleeping again....

In [ ]:
client_notifications = client_fiface.get_notifications()
client_notifications

### Now shutdown

In [ ]:
time.sleep(1)
await client_fiface.command_response(FilterCmdW("shutdown", graceful=True))